# ERA5‑Land · Temperatura del aire a 2 m (t2m) en puntos de Colombia

Este cuaderno descarga **temperatura del aire a 2 m** (t2m) desde **ERA5‑Land** (dataset horario) para uno o varios puntos,
y construye un **DataFrame** con las series en **°C**. Puedes elegir **agregación diaria** o **horaria**.

**Notas**
- ERA5‑Land es un **reanálisis** (malla ~9 km). No es satélite directo. Ideal para series continuas.
- El tiempo de ERA5 está en **UTC**; abajo añadimos también columna en hora local **America/Bogota**.


In [1]:
# Instalar y autenticar Google Earth Engine (solo la primera vez)
!pip -q install earthengine-api geemap pandas
import ee, geemap
ee.Authenticate()  # sigue el link, autoriza, pega el token
ee.Initialize()


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



Successfully saved authorization token.


In [ ]:
from datetime import datetime
import ee
import pandas as pd
from pathlib import Path

# ee.Initialize()  # ← Asegúrate de inicializar Earth Engine

# ---------------- Configuración ----------------
LOCATIONS = [
    {"name": "CEN-TRAF", "lat": 6.252306, "lon": -75.568852},
    {"name": "ITA-CJUS", "lat": 6.186118, "lon": -75.597457},
    {"name": "GIR-EPM",  "lat": 6.373513, "lon": -75.448027},
    {"name": "MED-FISC", "lat": 6.267655, "lon": -75.574471},
]

DATE_START = "2021-01-01"  # inclusive
DATE_END   = "2025-02-28"  # exclusiva en filterDate (usa +1 día si quieres incluir el fin exacto)

# "hourly" para datos por hora, "daily" para promedio diario
AGG = "hourly"  # "daily" o "hourly"

# ------------------------------------------------
COL = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate(DATE_START, DATE_END)

def hourly_ic(ic, band="temperature_2m"):
    def to_celsius(img):
        t = img.select([band]).subtract(273.15).rename("t2m_c")
        return t.set({
            "datetime_utc": img.date().format("YYYY-MM-dd HH:mm"),
            "system:time_start": img.get("system:time_start")
        })
    return ic.map(to_celsius)

def daily_means_from_hourly(ic, band="temperature_2m"):
    start = ee.Date(DATE_START)
    end   = ee.Date(DATE_END)
    n_days = end.difference(start, "day").toInt()

    def day_img(day_idx):
        d = start.advance(day_idx, "day")
        d1 = d.advance(1, "day")
        daily = ic.filterDate(d, d1).select([band]).mean().subtract(273.15).rename("t2m_c")
        return daily.set({
            "date": d.format("YYYY-MM-dd"),
            "system:time_start": d.millis()
        })
    days = ee.List.sequence(0, n_days.subtract(1))
    return ee.ImageCollection(days.map(day_img))

ic_use = daily_means_from_hourly(COL) if AGG == "daily" else hourly_ic(COL)

# ---- Puntos ----
features = []
for p in LOCATIONS:
    pt = ee.Geometry.Point([p["lon"], p["lat"]])
    features.append(ee.Feature(pt, {"name": p["name"], "lat": p["lat"], "lon": p["lon"]}))
points_fc = ee.FeatureCollection(features)

# ---- Muestreo en puntos para cada imagen ----
def sample_points(img):
    sampled = img.sampleRegions(
        collection=points_fc,
        properties=["name", "lat", "lon"],
        scale=10000,   # ERA5-Land ~9 km
        geometries=False
    )
    return sampled.map(lambda f: f.set({
        "date": img.get("date"),
        "datetime_utc": img.get("datetime_utc"),
        "time_start": img.get("system:time_start")
    }))

fc = ic_use.map(sample_points).flatten()

# ---- A pandas ----
_EXPECTED_COLS = ["name", "lat", "lon", "t2m_c", "date", "datetime_utc", "time_start"]

def fc_to_pandas(fc: ee.FeatureCollection, properties=None, limit=100000):
    size = int(fc.size().getInfo())
    size = min(size, limit)
    if size == 0:
        return pd.DataFrame()

    if properties is None:
        first_props = ee.Feature(fc.first()).toDictionary().keys().getInfo()
        properties = [c for c in _EXPECTED_COLS if c in first_props] or first_props

    flist = fc.limit(size).toList(size)
    dicts = flist.map(lambda f: ee.Feature(f).toDictionary(properties))
    data = dicts.getInfo()  # ⚠️ Si es muy grande, usa Export.table.toDrive
    return pd.DataFrame(data)

df = fc_to_pandas(
    fc,
    properties=["name", "lat", "lon", "t2m_c", "date", "datetime_utc", "time_start"],
)

# ===== Normalización de tiempos (una sola vez) =====
# Preferimos datetime_utc (string) → UTC; si no, 'date'; si no, 'time_start' (ms)
if "datetime_utc" in df.columns and df["datetime_utc"].notna().any():
    dt_utc = pd.to_datetime(df["datetime_utc"], format="%Y-%m-%d %H:%M", errors="coerce", utc=True)
elif "date" in df.columns and df["date"].notna().any():
    dt_utc = pd.to_datetime(df["date"], format="%Y-%m-%d", errors="coerce", utc=True)
else:
    # fallback a time_start en ms
    dt_utc = pd.to_datetime(df["time_start"], unit="ms", utc=True)

df["datetime_utc"] = dt_utc
df["datetime_bogota"] = df["datetime_utc"].dt.tz_convert("America/Bogota")

# Filas ordenadas y columnas básicas
df = df[["name","lat","lon","t2m_c","datetime_utc","datetime_bogota"]].dropna(subset=["datetime_utc"])
df = df.sort_values(["name","datetime_utc"]).reset_index(drop=True)

# ===== Pivot a formato ancho: una columna por punto =====
# Si hay duplicados por (name, datetime_utc), usamos la media.
wide_temp = (
    df.pivot_table(index="datetime_utc", columns="name", values="t2m_c", aggfunc="mean")
    .sort_index()
)

# Añadimos columnas de tiempo al principio (UTC + Bogotá)
df_out = wide_temp.copy()
df_out.insert(0, "datetime_bogota", df_out.index.tz_convert("America/Bogota"))
df_out.insert(0, "datetime_utc", df_out.index)

# (Opcional) convertir timestamps a ISO string para CSV "limpio"
df_out["datetime_utc"] = df_out["datetime_utc"].dt.strftime("%Y-%m-%d %H:%M:%S%z").str.replace(r"(\+00:00)$","Z", regex=True)
df_out["datetime_bogota"] = pd.to_datetime(df_out["datetime_bogota"]).dt.strftime("%Y-%m-%d %H:%M:%S%z")

# ===== Guardar a CSV =====
out_dir = Path("data/raw/era5")
out_dir.mkdir(parents=True, exist_ok=True)
# Si quieres que DATE_END sea inclusivo en el nombre, ajusta aquí.
fname = f"era5_t2m_{AGG}_{DATE_START}_{DATE_END}.csv"
out_path = out_dir / fname
df_out.to_csv(out_path, index=False)
print(f"CSV guardado en: {out_path.resolve()}")

# (Opcional) vista rápida
print(df_out.head(3))
print("shape:", df_out.shape)


In [8]:
import ee, pandas as pd
import math
ee.Initialize(project=None)

# ================== CONFIG ==================
LOCATIONS = [
    {"name": "CEN-TRAF", "lat": 6.252306, "lon": -75.568852},
    {"name": "ITA-CJUS", "lat": 6.186118, "lon": -75.597457},
    {"name": "GIR-EPM",  "lat": 6.373513, "lon": -75.448027},
    {"name": "MED-FISC", "lat": 6.267655, "lon": -75.574471},
]
DATE_START   = "2021-01-01"
DATE_END_EXC = "2025-03-01"   # fin exclusivo
DRIVE_FOLDER = "era5_exports_so2_no_turb"  # <- nueva carpeta para esta corrida
CHUNK_FREQ   = "MS"           # mensual

COL = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate(DATE_START, DATE_END_EXC)

# --- Imagen con bandas convertidas y DERIVADAS (sin turbulencia: NO sshf/slhf) ---
def to_features(img):
    # Temperatura y humedad (°C)
    t2m  = img.select('temperature_2m').subtract(273.15).rename('t2m_c')
    td2m = img.select('dewpoint_temperature_2m').subtract(273.15).rename('td2m_c')

    # Presión (hPa)
    sp   = img.select('surface_pressure').divide(100).rename('sp_hpa')

    # Viento (m/s) y dirección (grados 0..360)
    u10  = img.select('u_component_of_wind_10m').rename('u10')
    v10  = img.select('v_component_of_wind_10m').rename('v10')
    ws   = u10.hypot(v10).rename('wind_speed')

    # ⬇️ FIX: usa math.pi (float) o ee.Number(180).divide(math.pi)
    wind_dir = u10.atan2(v10).multiply(180.0 / math.pi).rename('wind_dir_deg')
    wind_dir = wind_dir.where(wind_dir.lt(0), wind_dir.add(360))

    # Precipitación horaria (m -> mm/h)
    tp_h = img.select('total_precipitation_hourly').multiply(1000).rename('tp_mm_h')

    # Radiación (J/m²·h -> W/m²)
    ssrdh  = img.select('surface_solar_radiation_downwards_hourly').divide(3600).rename('ssrd_wm2')
    ssrnet = img.select('surface_net_solar_radiation_hourly').divide(3600).rename('ssrnet_wm2')
    strdh  = img.select('surface_thermal_radiation_downwards_hourly').divide(3600).rename('strd_wm2')
    strnet = img.select('surface_net_thermal_radiation_hourly').divide(3600).rename('strnet_wm2')

    # Humedad relativa (%)
    a = ee.Number(17.625); b = ee.Number(243.04)
    rh = (td2m.expression(
        '100 * exp((a*TD)/(b+TD) - (a*T)/(b+T))',
        {'a': a, 'b': b, 'TD': td2m, 'T': t2m}
    ).rename('rh_percent')).clamp(0, 100)

    return (t2m.addBands([td2m, rh,
                          u10, v10, ws, wind_dir,
                          sp,
                          tp_h,
                          ssrdh, ssrnet, strdh, strnet])
              .set({
                  'datetime_utc': img.date().format('YYYY-MM-dd HH:mm'),
                  'system:time_start': img.get('system:time_start')
              }))

ic_feats = COL.map(to_features)

# Puntos
features = [ee.Feature(ee.Geometry.Point([p["lon"], p["lat"]]), p) for p in LOCATIONS]
points_fc = ee.FeatureCollection(features)

def sample_points(img):
    props = ["name", "lat", "lon"]
    sampled = img.sampleRegions(collection=points_fc, properties=props, scale=10000, geometries=False)
    return sampled.map(lambda f: f.set({
        'datetime_utc': img.get('datetime_utc'),
        'time_start': img.get('system:time_start')
    }))

def export_chunks_to_drive():
    edges = pd.date_range(DATE_START, DATE_END_EXC, freq=CHUNK_FREQ)
    tasks = []
    selectors = [
        "name","lat","lon",
        # thermo & humidity
        "t2m_c","td2m_c","rh_percent",
        # wind
        "u10","v10","wind_speed","wind_dir_deg",
        # pressure
        "sp_hpa",
        # precip & radiation
        "tp_mm_h","ssrd_wm2","ssrnet_wm2","strd_wm2","strnet_wm2",
        # timestamps
        "datetime_utc","time_start"
    ]
    for i in range(len(edges)-1):
        s = edges[i].strftime("%Y-%m-%d")
        e = edges[i+1].strftime("%Y-%m-%d")
        fc = ic_feats.filterDate(s, e).map(sample_points).flatten()
        desc  = f"era5_so2_noTurb_hourly_{edges[i].strftime('%Y%m')}"
        fname = f"era5_so2_noTurb_hourly_{s}_to_{e}"
        task = ee.batch.Export.table.toDrive(
            collection=fc,
            description=desc,
            folder=DRIVE_FOLDER,
            fileNamePrefix=fname,
            fileFormat="CSV",
            selectors=selectors
        )
        task.start()
        tasks.append((desc, task.id))
        print(f"Lanzada: {desc} | id: {task.id} | {s} → {e}")
    return tasks

tasks = export_chunks_to_drive()
print("Tasks lanzadas:", len(tasks))


Lanzada: era5_so2_noTurb_hourly_202101 | id: GV6CZZWC4QONEJXOS5ZVQVMT | 2021-01-01 → 2021-02-01
Lanzada: era5_so2_noTurb_hourly_202102 | id: J52RYAJ34YFKXFZOF2XF7VFZ | 2021-02-01 → 2021-03-01
Lanzada: era5_so2_noTurb_hourly_202103 | id: P3X2UNVOMFSQABRD2ZTA37YS | 2021-03-01 → 2021-04-01
Lanzada: era5_so2_noTurb_hourly_202104 | id: CYCSNET2UGYT7AS7C7Z2X4WB | 2021-04-01 → 2021-05-01
Lanzada: era5_so2_noTurb_hourly_202105 | id: JC73ZGH534MDRLBFKSCWCKZA | 2021-05-01 → 2021-06-01
Lanzada: era5_so2_noTurb_hourly_202106 | id: MMTYOIY5DPAPNROHLFPGHGRL | 2021-06-01 → 2021-07-01
Lanzada: era5_so2_noTurb_hourly_202107 | id: OB3ZH266F7NVO7JE5PXCUJT5 | 2021-07-01 → 2021-08-01
Lanzada: era5_so2_noTurb_hourly_202108 | id: STGTWDDWXUKHJBTTDGJEOYMD | 2021-08-01 → 2021-09-01
Lanzada: era5_so2_noTurb_hourly_202109 | id: PIWMURUCBQMWX4RYOUWIE4LI | 2021-09-01 → 2021-10-01
Lanzada: era5_so2_noTurb_hourly_202110 | id: GK7FHQWNNPDN5Q4CHKQDAUMD | 2021-10-01 → 2021-11-01
Lanzada: era5_so2_noTurb_hourly_202111 |

In [10]:
# === Exportar 5 horas "faltantes" como un archivo extra ===
import ee, pandas as pd, math
ee.Initialize(project=None)

# ===== Ajusta estas dos marcas en UTC (fin es exclusivo) =====
EXTRA_START_UTC = "2025-03-01T00:00"   # <-- ejemplo
EXTRA_END_UTC   = "2025-03-01T05:00"   # <-- ejemplo (5 horas)

# PUNTOS (mismo set que usaste)
LOCATIONS = [
    {"name": "CEN-TRAF", "lat": 6.252306, "lon": -75.568852},
    {"name": "ITA-CJUS", "lat": 6.186118, "lon": -75.597457},
    {"name": "GIR-EPM",  "lat": 6.373513, "lon": -75.448027},
    {"name": "MED-FISC", "lat": 6.267655, "lon": -75.574471},
]

# Carpeta de Drive donde dejaste los mensuales
DRIVE_FOLDER = "era5_exports_so2_no_turb"  # la misma que ya usaste

# Colección
COL = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate(EXTRA_START_UTC, EXTRA_END_UTC)

def to_features(img):
    t2m  = img.select('temperature_2m').subtract(273.15).rename('t2m_c')
    td2m = img.select('dewpoint_temperature_2m').subtract(273.15).rename('td2m_c')
    sp   = img.select('surface_pressure').divide(100).rename('sp_hpa')
    u10  = img.select('u_component_of_wind_10m').rename('u10')
    v10  = img.select('v_component_of_wind_10m').rename('v10')
    ws   = u10.hypot(v10).rename('wind_speed')
    wind_dir = u10.atan2(v10).multiply(180.0 / math.pi).rename('wind_dir_deg')
    wind_dir = wind_dir.where(wind_dir.lt(0), wind_dir.add(360))
    tp_h  = img.select('total_precipitation_hourly').multiply(1000).rename('tp_mm_h')
    ssrdh = img.select('surface_solar_radiation_downwards_hourly').divide(3600).rename('ssrd_wm2')
    ssrnet = img.select('surface_net_solar_radiation_hourly').divide(3600).rename('ssrnet_wm2')
    strdh = img.select('surface_thermal_radiation_downwards_hourly').divide(3600).rename('strd_wm2')
    strnet = img.select('surface_net_thermal_radiation_hourly').divide(3600).rename('strnet_wm2')
    a = ee.Number(17.625); b = ee.Number(243.04)
    rh = (td2m.expression(
        '100 * exp((a*TD)/(b+TD) - (a*T)/(b+T))',
        {'a': a, 'b': b, 'TD': td2m, 'T': t2m}
    ).rename('rh_percent')).clamp(0, 100)

    return (t2m.addBands([td2m, rh, u10, v10, ws, wind_dir, sp, tp_h, ssrdh, ssrnet, strdh, strnet])
              .set({'datetime_utc': img.date().format('YYYY-MM-dd HH:mm'),
                    'system:time_start': img.get('system:time_start')}))

ic_feats = COL.map(to_features)

# Puntos
features = [ee.Feature(ee.Geometry.Point([p["lon"], p["lat"]]), p) for p in LOCATIONS]
points_fc = ee.FeatureCollection(features)

def sample_points(img):
    props = ["name","lat","lon"]
    sampled = img.sampleRegions(collection=points_fc, properties=props, scale=10000, geometries=False)
    return sampled.map(lambda f: f.set({
        'datetime_utc': img.get('datetime_utc'),
        'time_start': img.get('system:time_start')
    }))

selectors = [
    "name","lat","lon",
    "t2m_c","td2m_c","rh_percent",
    "u10","v10","wind_speed","wind_dir_deg",
    "sp_hpa",
    "tp_mm_h","ssrd_wm2","ssrnet_wm2","strd_wm2","strnet_wm2",
    "datetime_utc","time_start"
]

# Export único del rango corto
fc_extra = ic_feats.map(sample_points).flatten()
desc  = f"era5_so2_noTurb_hourly_EXTRA_{EXTRA_START_UTC.replace(':','').replace('-','')}__{EXTRA_END_UTC.replace(':','').replace('-','')}"
fname = f"era5_so2_noTurb_hourly_{EXTRA_START_UTC}_to_{EXTRA_END_UTC}"

task = ee.batch.Export.table.toDrive(
    collection=fc_extra,
    description=desc,
    folder=DRIVE_FOLDER,
    fileNamePrefix=fname,
    fileFormat="CSV",
    selectors=selectors
)
task.start()
print("Task lanzada:", desc)


Task lanzada: era5_so2_noTurb_hourly_EXTRA_20250301T0000__20250301T0500


In [11]:
from pathlib import Path
import pandas as pd
import numpy as np

# ========= CONFIG =========
# Carpeta donde guardaste TODOS los CSV descargados de Drive (mensuales)
IN_DIR  = Path("era5_exports")   # <-- cámbiala si aplica
OUT_DIR = Path("data/raw/era5")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Estaciones en el orden exacto que quieres en el header final
# Nota: si tus CSV tienen "GIR-EPM" pero quieres "V-GIR-EPM" en el encabezado, lo renombramos.
stations_order_final = ["V-GIR-EPM", "MED-FISC", "CEN-TRAF", "ITA-CJUS"]

# Renombrado de estaciones (como vienen en los CSV -> como quieres en el header)
STATION_RENAME = {
    "GIR-EPM": "V-GIR-EPM",
    # deja idénticas las demás si coinciden
    "MED-FISC": "MED-FISC",
    "CEN-TRAF": "CEN-TRAF",
    "ITA-CJUS": "ITA-CJUS",
}

# Variables que exportaste (sin turbulencia)
# y cómo se llaman en tu header deseado (nomenclatura del usuario)
labels_order = ["DViento_SSR","HAire10_SSR","PLiquida_SSR","P_SSR","TAire10_SSR","VViento_SSR"]
label_to_col = {
    "DViento_SSR": "wind_dir_deg",   # grados (0..360)
    "HAire10_SSR": "rh_percent",     # %
    "PLiquida_SSR": "tp_mm_h",       # mm/h
    "P_SSR":       "sp_hpa",         # hPa
    "TAire10_SSR": "t2m_c",          # °C
    "VViento_SSR": "wind_speed",     # m/s
}
VARS = list(label_to_col.values())

# ========= 1) LEER Y CONCATENAR (LARGO) =========
files = sorted([p for p in IN_DIR.glob("*.csv") if p.is_file()])
if not files:
    raise SystemExit(f"No se encontraron CSV en {IN_DIR.resolve()}")

parts = []
for p in files:
    df = pd.read_csv(p)
    parts.append(df)

long_df = pd.concat(parts, ignore_index=True)

# ========= 2) NORMALIZAR TIEMPO =========
# Preferimos datetime_utc si existe y trae datos; si no, usamos time_start (ms)
if "datetime_utc" in long_df.columns and long_df["datetime_utc"].notna().any():
    long_df["datetime_utc"] = pd.to_datetime(long_df["datetime_utc"], errors="coerce", utc=True)
else:
    if "time_start" not in long_df.columns:
        raise SystemExit("No hay columnas 'datetime_utc' ni 'time_start' en los CSV.")
    long_df["datetime_utc"] = pd.to_datetime(long_df["time_start"], unit="ms", utc=True)

# Renombrar estaciones si hace falta (GIR-EPM -> V-GIR-EPM)
if "name" not in long_df.columns:
    raise SystemExit("No se encontró la columna 'name' (estación) en los CSV.")
long_df["name"] = long_df["name"].map(lambda s: STATION_RENAME.get(s, s))

# ========= 3) LIMPIEZA Y AGREGACIÓN BÁSICA =========
# Asegurar numeric de las variables
for c in VARS:
    if c in long_df.columns:
        long_df[c] = pd.to_numeric(long_df[c], errors="coerce")

# Nos quedamos con lo necesario
keep_cols = ["name","datetime_utc"] + [c for c in VARS if c in long_df.columns]
long_df = long_df[keep_cols].dropna(subset=["datetime_utc"]).copy()

# Si hay duplicados (name, datetime_utc) promediamos
agg_dict = {v: "mean" for v in VARS if v in long_df.columns}
long_df = (long_df
           .groupby(["datetime_utc","name"], as_index=False, sort=True)
           .agg(agg_dict)
           .sort_values(["name","datetime_utc"])
           .reset_index(drop=True))

# ========= 4) CONSTRUIR MATRIZ (index=tiempo, columnas=(estación, variable)) =========
all_ts = long_df["datetime_utc"].dropna().sort_values().unique()
wide = pd.DataFrame(index=pd.DatetimeIndex(all_ts))

# rellenar en el orden solicitado
for st in stations_order_final:
    sdf = long_df[long_df["name"] == st].set_index("datetime_utc").sort_index()
    for lab in labels_order:
        col = label_to_col[lab]
        series = sdf[col].reindex(wide.index) if col in sdf.columns else pd.Series(index=wide.index, dtype="float64")
        wide[(st, lab)] = series

# asegurar orden exacto de columnas
wide = wide.reindex(columns=pd.MultiIndex.from_product([stations_order_final, labels_order]), copy=False)

# ========= 5) EXPORTAR CON DOBLE ENCABEZADO PERSONALIZADO =========
out_csv = OUT_DIR / "era5_SO2_hourly_custom_header_2021-01-01_2025-02-28.csv"
out_csv.parent.mkdir(parents=True, exist_ok=True)

# Filas 1 y 2 del encabezado
row1 = ["est"] + [st for st in stations_order_final for _ in labels_order]
row2 = [""]    + [lab for _ in stations_order_final for lab in labels_order]

# Escribir CSV
with out_csv.open("w", encoding="utf-8") as f:
    # encabezados
    f.write(",".join(row1) + "\n")
    f.write(",".join(row2) + "\n")
    # datos (datetime en America/Bogota)
    for ts, row in wide.iterrows():
        ts_str = ts.tz_convert("America/Bogota").strftime("%Y-%m-%d %H:%M:%S")
        vals = []
        for st in stations_order_final:
            for lab in labels_order:
                v = row[(st, lab)]
                vals.append("" if pd.isna(v) else f"{v:.3f}".rstrip("0").rstrip("."))
        f.write(",".join([ts_str] + vals) + "\n")

print("✔ CSV generado:", out_csv.resolve())
print("shape:", (len(wide.index), len(row1)-1), "| filas (horas) x columnas (estación*variable)")


✔ CSV generado: C:\Maestria\Trabajo de grado\repositorio\data\raw\era5\era5_SO2_hourly_custom_header_2021-01-01_2025-02-28.csv
shape: (36485, 24) | filas (horas) x columnas (estación*variable)


In [5]:
import pandas as pd
from pathlib import Path
# Ruta donde pusiste los CSV descargados
IN_DIR = Path("era5_exports")
dfs = [pd.read_csv(p) for p in sorted(IN_DIR.glob("era5_t2m_hourly_*.csv"))]
df = pd.concat(dfs, ignore_index=True)

# Normaliza tiempo (UTC preferido)
if df["datetime_utc"].notna().any():
    dt_utc = pd.to_datetime(df["datetime_utc"], format="%Y-%m-%d %H:%M", errors="coerce", utc=True)
elif "date" in df.columns and df["date"].notna().any():
    dt_utc = pd.to_datetime(df["date"], format="%Y-%m-%d", errors="coerce", utc=True)
else:
    dt_utc = pd.to_datetime(df["time_start"], unit="ms", utc=True)

df["datetime_utc"]   = dt_utc
df["datetime_bogota"] = df["datetime_utc"].dt.tz_convert("America/Bogota")

wide = (
    df.pivot_table(index="datetime_utc", columns="name", values="t2m_c", aggfunc="mean")
      .sort_index()
)
out = wide.copy()
out.insert(0, "datetime_bogota", out.index.tz_convert("America/Bogota"))
out.insert(0, "datetime_utc", out.index)

out["datetime_utc"] = out["datetime_utc"].dt.strftime("%Y-%m-%d %H:%M:%S%z").str.replace(r"(\+00:00)$","Z", regex=True)
out["datetime_bogota"] = pd.to_datetime(out["datetime_bogota"]).dt.strftime("%Y-%m-%d %H:%M:%S%z")

Path("data/raw/era5").mkdir(parents=True, exist_ok=True)
out.to_csv("data/raw/era5/era5_t2m_hourly_2021-01-01_2025-02-28_wide.csv", index=False)
print(out.head())


name                                   datetime_utc           datetime_bogota  \
datetime_utc                                                                    
2021-01-01 00:00:00+00:00  2021-01-01 00:00:00+0000  2020-12-31 19:00:00-0500   
2021-01-01 01:00:00+00:00  2021-01-01 01:00:00+0000  2020-12-31 20:00:00-0500   
2021-01-01 02:00:00+00:00  2021-01-01 02:00:00+0000  2020-12-31 21:00:00-0500   
2021-01-01 03:00:00+00:00  2021-01-01 03:00:00+0000  2020-12-31 22:00:00-0500   
2021-01-01 04:00:00+00:00  2021-01-01 04:00:00+0000  2020-12-31 23:00:00-0500   

name                        CEN-TRAF    GIR-EPM   ITA-CJUS   MED-FISC  
datetime_utc                                                           
2021-01-01 00:00:00+00:00  16.558725  15.757944  15.599741  16.558725  
2021-01-01 01:00:00+00:00  15.368188  14.704126  14.686548  15.368188  
2021-01-01 02:00:00+00:00  14.764734  14.006921  14.071375  14.764734  
2021-01-01 03:00:00+00:00  14.629373  13.519998  13.830545  14.629373  


In [6]:
ic = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
first = ic.first()
print(first.bandNames().getInfo())


['dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature', 'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4', 'lake_bottom_temperature', 'lake_ice_depth', 'lake_ice_temperature', 'lake_mix_layer_depth', 'lake_mix_layer_temperature', 'lake_shape_factor', 'lake_total_layer_temperature', 'snow_albedo', 'snow_cover', 'snow_density', 'snow_depth', 'snow_depth_water_equivalent', 'snowfall', 'snowmelt', 'temperature_of_snow_layer', 'skin_reservoir_content', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'forecast_albedo', 'surface_latent_heat_flux', 'surface_net_solar_radiation', 'surface_net_thermal_radiation', 'surface_sensible_heat_flux', 'surface_solar_radiation_downwards', 'surface_thermal_radiation_downwards', 'evaporation_from_bare_soil', 'evaporation_from_open_water_surfaces_excluding_oceans', 'evaporation_from_the_top_of_canopy', 'ev

## Documentación rápida
- **Dataset**: `ECMWF/ERA5_LAND/HOURLY` (t2m en Kelvin → convertimos a **°C**).
- **AGG**:
  - `hourly`: mantiene las horas originales.
  - `daily`: promedia por día con `mean()` en GEE.
- **`sampleRegions`**: toma el valor del píxel ERA5‑Land (~9 km) más cercano al punto.
- **Tiempo**: el dataset está en **UTC**. Se agrega `datetime_local` en **America/Bogota**.
- **Salida**: `era5land_t2m_points.csv` con columnas (`name, lat, lon, date/datetime_utc, datetime_local, t2m_c`).

### Cambios típicos
- Añadir más ubicaciones a `LOCATIONS`.
- Cambiar `DATE_START`, `DATE_END` y `AGG`.
- Reemplazar `scale=10000` por otro valor si prefieres muestreo a una escala ligeramente distinta.
